# Multi-Subject Data Aggregator v4.13_cc

This notebook loads aggregated subject CSV files and merges them into a single mega_df for large-scale cross-subject analysis.

## Workflow:
1. **Scan** - Find all aggregated CSV files (e.g., `CA_Cvc_25.11.19.csv`, `CI101_Cvc_25.11.19.csv`)
2. **Load** - Read each CSV and add subject_id column
3. **Merge** - Combine into mega_df
4. **Analyze** - Use existing analysis tools for cross-subject comparisons

## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import ttest_ind, mannwhitneyu, shapiro, levene, pearsonr, spearmanr
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import os
import re
import glob
from pathlib import Path
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

# Plot settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('Set2')

print("Libraries imported successfully")

## 2. CSV File Scanner

Scans specified directories for aggregated subject CSV files.

In [ ]:
def scan_subject_csvs(base_directory, pattern='*_C*c_*.csv'):
    """
    Scans directory tree for aggregated subject CSV files.
    
    Parameters:
    -----------
    base_directory : str or Path
        Root directory to search for CSV files
    pattern : str
        Glob pattern to match CSV filenames (default: '*_C*c_*.csv')
        Examples: 'CA_Cvc_25.11.19.csv', 'CI101_CVC_25.11.19.csv'
    
    Returns:
    --------
    list of dict
        List containing {subject_id, filepath} for each found file
    """
    base_path = Path(base_directory)
    
    if not base_path.exists():
        print(f"Error: Directory '{base_directory}' does not exist")
        return []
    
    # Find all matching CSV files recursively
    csv_files = list(base_path.rglob(pattern))
    
    if not csv_files:
        print(f"No CSV files found matching pattern '{pattern}' in {base_directory}")
        return []
    
    # Extract subject IDs from filenames
    results = []
    for csv_path in csv_files:
        # Extract subject_id from filename (before first underscore)
        filename = csv_path.name
        subject_id = filename.split('_')[0]
        
        results.append({
            'subject_id': subject_id,
            'filename': filename,
            'filepath': csv_path
        })
    
    # Sort by subject_id
    results.sort(key=lambda x: x['subject_id'])
    
    print(f"\n{'='*70}")
    print(f"Found {len(results)} subject CSV files")
    print(f"{'='*70}")
    for r in results:
        print(f"  {r['subject_id']:10s} -> {r['filename']}")
    print(f"{'='*70}\n")
    
    return results

# Test the scanner (update path as needed)
# found_files = scan_subject_csvs('/path/to/your/data/directory')

## 3. Multi-Subject CSV Loader

Loads all subject CSV files and merges them into a single DataFrame.

In [ ]:
def load_and_merge_subjects(base_directory, pattern='*_C*c_*.csv', verbose=True):
    """
    Loads all subject CSV files and merges into single mega_df.
    
    Parameters:
    -----------
    base_directory : str or Path
        Root directory containing subject CSV files
    pattern : str
        Glob pattern for CSV files
    verbose : bool
        Print loading progress
    
    Returns:
    --------
    pd.DataFrame
        Merged dataframe with all subjects (includes 'subject_id' column)
    """
    # Scan for CSV files
    found_files = scan_subject_csvs(base_directory, pattern)
    
    if not found_files:
        return pd.DataFrame()
    
    # Load each CSV and add subject_id
    all_data = []
    load_stats = []
    
    if verbose:
        print("Loading subject data...")
        print(f"{'='*70}")
    
    for file_info in found_files:
        subject_id = file_info['subject_id']
        filepath = file_info['filepath']
        
        try:
            # Load CSV
            df = pd.read_csv(filepath)
            
            # Add subject_id column
            df['subject_id'] = subject_id
            
            # Track data types present
            tasks_present = df['task'].unique() if 'task' in df.columns else []
            
            all_data.append(df)
            
            load_stats.append({
                'subject_id': subject_id,
                'n_trials': len(df),
                'tasks': ', '.join(tasks_present)
            })
            
            if verbose:
                print(f"  {subject_id:10s} | {len(df):5d} trials | {', '.join(tasks_present)}")
        
        except Exception as e:
            print(f"  ERROR loading {subject_id}: {e}")
            continue
    
    if verbose:
        print(f"{'='*70}\n")
    
    # Merge all subjects
    if all_data:
        mega_df = pd.concat(all_data, ignore_index=True, sort=False)
        
        if verbose:
            print(f"\n{'='*70}")
            print(f"MEGA_DF CREATED")
            print(f"{'='*70}")
            print(f"Total subjects: {mega_df['subject_id'].nunique()}")
            print(f"Total trials: {len(mega_df):,}")
            if 'task' in mega_df.columns:
                print(f"\nTask breakdown:")
                for task, count in mega_df['task'].value_counts().items():
                    print(f"  {task:12s}: {count:6,} trials ({count/len(mega_df)*100:.1f}%)")
            print(f"{'='*70}\n")
        
        return mega_df
    else:
        print("No data loaded")
        return pd.DataFrame()

# Example usage:
# mega_df = load_and_merge_subjects('/path/to/your/data/directory')

## 4. Quick Data Explorer

Utility functions to explore the merged dataset.

In [ ]:
def explore_mega_df(mega_df):
    """
    Prints comprehensive overview of the mega_df.
    """
    if mega_df.empty:
        print("DataFrame is empty")
        return
    
    print(f"\n{'='*70}")
    print(f"MEGA_DF OVERVIEW")
    print(f"{'='*70}")
    print(f"Shape: {mega_df.shape[0]:,} rows x {mega_df.shape[1]} columns")
    print(f"Memory: {mega_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # Subject breakdown
    print(f"\nSubjects ({mega_df['subject_id'].nunique()}):")
    subject_counts = mega_df['subject_id'].value_counts().sort_index()
    for subj, count in subject_counts.items():
        print(f"  {subj:10s}: {count:5,} trials")
    
    # Task breakdown
    if 'task' in mega_df.columns:
        print(f"\nTasks:")
        for task, count in mega_df['task'].value_counts().items():
            print(f"  {task:12s}: {count:6,} trials ({count/len(mega_df)*100:.1f}%)")
    
    # Column types
    print(f"\nColumn types:")
    print(f"  Numeric:  {mega_df.select_dtypes(include=[np.number]).shape[1]}")
    print(f"  Object:   {mega_df.select_dtypes(include=['object']).shape[1]}")
    print(f"  Other:    {mega_df.shape[1] - mega_df.select_dtypes(include=[np.number, 'object']).shape[1]}")
    
    # Missing data
    missing = mega_df.isnull().sum()
    missing = missing[missing > 0].sort_values(ascending=False)
    if len(missing) > 0:
        print(f"\nColumns with missing data (top 10):")
        for col, count in missing.head(10).items():
            pct = count / len(mega_df) * 100
            print(f"  {col:20s}: {count:6,} ({pct:5.1f}%)")
    
    print(f"\nColumn names:")
    for i, col in enumerate(mega_df.columns, 1):
        print(f"  {i:2d}. {col}")
    
    print(f"{'='*70}\n")

# Example usage:
# explore_mega_df(mega_df)

## 5. Task-Specific Data Extractors

Split mega_df by task type for focused analysis.

In [ ]:
def split_by_task(mega_df):
    """
    Splits mega_df into separate DataFrames by task type.
    
    Returns:
    --------
    dict
        Dictionary with keys: 'Consonants', 'Vowels', 'CRM'
    """
    if 'task' not in mega_df.columns:
        print("Warning: 'task' column not found in mega_df")
        return {}
    
    result = {}
    
    for task in mega_df['task'].unique():
        df_task = mega_df[mega_df['task'] == task].copy()
        result[task] = df_task
        print(f"{task:12s}: {len(df_task):6,} trials, {df_task['subject_id'].nunique()} subjects")
    
    return result

def get_subjects_by_task(mega_df, min_trials=10):
    """
    Shows which subjects have data for each task.
    
    Parameters:
    -----------
    min_trials : int
        Minimum trials required to count subject as having task data
    """
    if 'task' not in mega_df.columns:
        return
    
    tasks = mega_df['task'].unique()
    subjects = sorted(mega_df['subject_id'].unique())
    
    print(f"\n{'='*70}")
    print(f"Subject x Task Matrix (min {min_trials} trials)")
    print(f"{'='*70}")
    print(f"{'Subject':10s} | ", end='')
    for task in tasks:
        print(f"{task:12s} | ", end='')
    print()
    print("-" * 70)
    
    for subj in subjects:
        print(f"{subj:10s} | ", end='')
        for task in tasks:
            n = len(mega_df[(mega_df['subject_id'] == subj) & (mega_df['task'] == task)])
            if n >= min_trials:
                print(f"{n:6d} trials | ", end='')
            else:
                print(f"{'---':>12s} | ", end='')
        print()
    print(f"{'='*70}\n")

# Example usage:
# task_dfs = split_by_task(mega_df)
# df_consonants = task_dfs['Consonants']
# df_vowels = task_dfs['Vowels']
# df_crm = task_dfs['CRM']
# get_subjects_by_task(mega_df)

## 6. Cross-Subject Comparison Tools

Analysis functions for comparing metrics across subjects.

In [ ]:
def compare_subjects_by_metric(mega_df, metric='score', task_filter=None, 
                                plot=True, figsize=(12, 6)):
    """
    Compares subjects on a specific metric (e.g., accuracy, RT, score).
    
    Parameters:
    -----------
    metric : str
        Column name to analyze (e.g., 'score', 'rt', 'correct', 'snr')
    task_filter : str or None
        Filter to specific task ('Consonants', 'Vowels', 'CRM') or None for all
    plot : bool
        Whether to create visualization
    """
    # Filter by task if specified
    if task_filter and 'task' in mega_df.columns:
        df = mega_df[mega_df['task'] == task_filter].copy()
        title_suffix = f" ({task_filter})"
    else:
        df = mega_df.copy()
        title_suffix = " (All Tasks)"
    
    if metric not in df.columns:
        print(f"Error: Column '{metric}' not found in data")
        return
    
    # Calculate summary stats per subject
    summary = df.groupby('subject_id')[metric].agg([
        ('mean', 'mean'),
        ('std', 'std'),
        ('median', 'median'),
        ('n', 'count')
    ]).round(3)
    
    summary = summary.sort_values('mean', ascending=False)
    
    print(f"\n{'='*70}")
    print(f"Subject Comparison: {metric}{title_suffix}")
    print(f"{'='*70}")
    print(summary)
    print(f"\nOverall mean: {df[metric].mean():.3f}")
    print(f"Overall std:  {df[metric].std():.3f}")
    print(f"{'='*70}\n")
    
    # Plot if requested
    if plot:
        fig, axes = plt.subplots(1, 2, figsize=figsize)
        
        # Bar plot of means
        ax1 = axes[0]
        summary['mean'].plot(kind='bar', ax=ax1, color='steelblue', alpha=0.7)
        ax1.set_title(f'Mean {metric} by Subject{title_suffix}')
        ax1.set_xlabel('Subject ID')
        ax1.set_ylabel(f'Mean {metric}')
        ax1.axhline(df[metric].mean(), color='red', linestyle='--', 
                    label='Overall Mean', alpha=0.7)
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45, ha='right')
        
        # Box plot distribution
        ax2 = axes[1]
        df.boxplot(column=metric, by='subject_id', ax=ax2)
        ax2.set_title(f'{metric} Distribution by Subject{title_suffix}')
        ax2.set_xlabel('Subject ID')
        ax2.set_ylabel(metric)
        plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45, ha='right')
        plt.suptitle('')  # Remove default title
        
        plt.tight_layout()
        plt.show()
    
    return summary

# Example usage:
# compare_subjects_by_metric(mega_df, metric='score', task_filter='Consonants')
# compare_subjects_by_metric(mega_df, metric='rt', task_filter='Vowels')

## 7. Subject Group Analysis

Tools for analyzing subject groups (e.g., CI vs HS).

In [ ]:
def categorize_subjects(mega_df, custom_categories=None):
    """
    Adds 'subject_group' column based on subject_id prefix.
    
    Parameters:
    -----------
    custom_categories : dict or None
        Custom mapping of patterns to group names
        Example: {'CI': 'Cochlear Implant', 'HS': 'Hearing', 'CA': 'Control'}
    
    Returns:
    --------
    pd.DataFrame
        mega_df with added 'subject_group' column
    """
    df = mega_df.copy()
    
    if custom_categories is None:
        # Default categories
        def assign_group(subject_id):
            if subject_id.startswith('CI'):
                return 'CI'
            elif subject_id.startswith('HS'):
                return 'HS'
            elif subject_id.startswith('CA'):
                return 'CA'
            elif subject_id.startswith('LR'):
                return 'LR'
            else:
                return 'Other'
    else:
        def assign_group(subject_id):
            for pattern, group in custom_categories.items():
                if subject_id.startswith(pattern):
                    return group
            return 'Other'
    
    df['subject_group'] = df['subject_id'].apply(assign_group)
    
    # Print summary
    print(f"\n{'='*70}")
    print(f"Subject Groups")
    print(f"{'='*70}")
    for group in sorted(df['subject_group'].unique()):
        subjects = df[df['subject_group'] == group]['subject_id'].unique()
        n_trials = len(df[df['subject_group'] == group])
        print(f"{group:15s}: {len(subjects):2d} subjects, {n_trials:6,} trials")
        print(f"                 {', '.join(sorted(subjects))}")
    print(f"{'='*70}\n")
    
    return df

def compare_groups_by_metric(mega_df, metric='score', task_filter=None, 
                             plot=True, stats_test=True):
    """
    Compares subject groups on a metric with statistical testing.
    
    Requires 'subject_group' column (use categorize_subjects first).
    """
    if 'subject_group' not in mega_df.columns:
        print("Error: Run categorize_subjects() first to create subject_group column")
        return
    
    # Filter by task
    if task_filter and 'task' in mega_df.columns:
        df = mega_df[mega_df['task'] == task_filter].copy()
        title_suffix = f" ({task_filter})"
    else:
        df = mega_df.copy()
        title_suffix = ""
    
    if metric not in df.columns:
        print(f"Error: Column '{metric}' not found")
        return
    
    # Summary by group
    summary = df.groupby('subject_group')[metric].agg([
        ('mean', 'mean'),
        ('std', 'std'),
        ('median', 'median'),
        ('n', 'count')
    ]).round(3)
    
    print(f"\n{'='*70}")
    print(f"Group Comparison: {metric}{title_suffix}")
    print(f"{'='*70}")
    print(summary)
    print(f"{'='*70}\n")
    
    # Statistical testing
    if stats_test:
        groups = df['subject_group'].unique()
        if len(groups) == 2:
            g1, g2 = groups
            data1 = df[df['subject_group'] == g1][metric].dropna()
            data2 = df[df['subject_group'] == g2][metric].dropna()
            
            # Mann-Whitney U test (non-parametric)
            stat, p = mannwhitneyu(data1, data2, alternative='two-sided')
            print(f"Mann-Whitney U Test:")
            print(f"  {g1} vs {g2}")
            print(f"  U-statistic = {stat:.3f}, p-value = {p:.4f}")
            if p < 0.05:
                print(f"  Result: Significant difference (p < 0.05)")
            else:
                print(f"  Result: No significant difference (p >= 0.05)")
            print()
    
    # Plot
    if plot:
        fig, ax = plt.subplots(1, 1, figsize=(10, 6))
        df.boxplot(column=metric, by='subject_group', ax=ax)
        ax.set_title(f'{metric} by Subject Group{title_suffix}')
        ax.set_xlabel('Subject Group')
        ax.set_ylabel(metric)
        plt.suptitle('')
        plt.tight_layout()
        plt.show()
    
    return summary

# Example usage:
# mega_df = categorize_subjects(mega_df)
# compare_groups_by_metric(mega_df, metric='score', task_filter='Consonants')

## 8. Export Utilities

Save processed mega_df for future use.

In [ ]:
def save_mega_df(mega_df, output_path='mega_df_all_subjects.csv', 
                 include_timestamp=True):
    """
    Saves mega_df to CSV file.
    
    Parameters:
    -----------
    output_path : str
        Output filename
    include_timestamp : bool
        Add timestamp to filename
    """
    from datetime import datetime
    
    if include_timestamp:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        base, ext = os.path.splitext(output_path)
        output_path = f"{base}_{timestamp}{ext}"
    
    mega_df.to_csv(output_path, index=False)
    
    file_size = os.path.getsize(output_path) / 1024**2
    
    print(f"\n{'='*70}")
    print(f"MEGA_DF SAVED")
    print(f"{'='*70}")
    print(f"File: {output_path}")
    print(f"Size: {file_size:.2f} MB")
    print(f"Rows: {len(mega_df):,}")
    print(f"Cols: {len(mega_df.columns)}")
    print(f"{'='*70}\n")

def load_saved_mega_df(filepath):
    """
    Loads a previously saved mega_df.
    """
    print(f"Loading {filepath}...")
    mega_df = pd.read_csv(filepath)
    print(f"Loaded {len(mega_df):,} rows, {len(mega_df.columns)} columns")
    return mega_df

# Example usage:
# save_mega_df(mega_df)
# mega_df = load_saved_mega_df('mega_df_all_subjects_20241120_143052.csv')

## 9. Main Execution Block

**Update the base_directory path and run this cell to load all data.**

In [ ]:
# ===== MAIN EXECUTION =====
# Update this path to your data directory containing the aggregated CSV files
BASE_DATA_DIR = '/home/user/Disco/Data'  # UPDATE THIS PATH

# Load and merge all subject data
mega_df = load_and_merge_subjects(
    base_directory=BASE_DATA_DIR,
    pattern='*_C*c_*.csv',  # Matches CA_Cvc_25.11.19.csv, CI101_CVC_25.11.19.csv, etc.
    verbose=True
)

# Explore the merged dataset
if not mega_df.empty:
    explore_mega_df(mega_df)
    
    # Add subject grouping
    mega_df = categorize_subjects(mega_df)
    
    # Show task distribution
    print("\nSubject x Task Matrix:")
    get_subjects_by_task(mega_df, min_trials=10)

## 10. Quick Analysis Examples

Example analyses using the loaded mega_df.

In [ ]:
# Example 1: Compare accuracy across subjects for Consonants
if not mega_df.empty and 'score' in mega_df.columns:
    compare_subjects_by_metric(mega_df, metric='score', task_filter='Consonants')

In [ ]:
# Example 2: Compare reaction times for Vowels
if not mega_df.empty and 'rt' in mega_df.columns:
    compare_subjects_by_metric(mega_df, metric='rt', task_filter='Vowels')

In [ ]:
# Example 3: Compare CI vs HS groups
if not mega_df.empty and 'subject_group' in mega_df.columns:
    compare_groups_by_metric(mega_df, metric='score', task_filter='Consonants')

In [ ]:
# Example 4: Split data by task for detailed analysis
if not mega_df.empty:
    task_dfs = split_by_task(mega_df)
    
    # Access individual task dataframes
    if 'Consonants' in task_dfs:
        df_consonants = task_dfs['Consonants']
        print(f"\nConsonants data: {len(df_consonants)} trials")
    
    if 'Vowels' in task_dfs:
        df_vowels = task_dfs['Vowels']
        print(f"Vowels data: {len(df_vowels)} trials")
    
    if 'CRM' in task_dfs:
        df_crm = task_dfs['CRM']
        print(f"CRM data: {len(df_crm)} trials")

In [ ]:
# Example 5: Save the mega_df for future use
if not mega_df.empty:
    save_mega_df(mega_df, output_path='mega_df_all_subjects.csv')